# **Cross-Validation**

In this Jupyter Notebook, we explore the fundamental concepts of splitting data into training and testing sets and applying cross-validation to evaluate model performance. We will use the `seaborn mpg` dataset to build a Linear Regression model that predicts a car's fuel mileage based on its characteristics. Throughout the notebook, we will demonstrate both manual and Scikit-learn methods for data splitting, model building, and evaluation. Additionally, we will introduce Scikit-learn's `Pipeline` for streamlined workflow management and discuss best practices to avoid common pitfalls like data leakage.

***

## **1. Import Libraries**

First, we import the necessary libraries for data manipulation, visualization, and modeling.

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import cufflinks as cf
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import CountVectorizer
from seaborn import load_dataset
# Configure Cufflinks for offline use with a ggplot theme
cf.set_config_file(offline=True, sharing=False, theme='ggplot')

***

## **2. The Data**

We will use the `mpg` dataset from `seaborn`, which provides information about the fuel mileage (`mpg`) of various cars along with their characteristics.

Our objective is to build a model that predicts a car's fuel mileage based on these features.

In [2]:
# Load the mpg dataset
data = load_dataset("mpg")

# Display the first few rows of the dataset
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


***

## **3. Train-Test Split**

Splitting the dataset into training and testing sets is crucial for evaluating the performance and generalizability of our model.

We will explore two methods to perform this split: using Pandas operations and Scikit-learn's `train_test_split` function.

### **3.1 ~ Using Pandas Operations**

We can manually shuffle the dataset and split it into training and testing sets.

In [3]:
# Shuffle the dataset
shuffled_data = data.sample(frac=1.0, random_state=42).reset_index(drop=True)

# Display the shuffled data
shuffled_data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,33.0,4,91.0,53.0,1795,17.4,76,japan,honda civic
1,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger
2,19.0,6,232.0,100.0,2634,13.0,71,usa,amc gremlin
3,13.0,8,318.0,150.0,3940,13.2,76,usa,plymouth volare premier v8
4,14.0,8,318.0,150.0,4237,14.5,73,usa,plymouth fury gran sedan


In [4]:
# Determine the split point (90% training, 10% testing)
split_point = int(shuffled_data.shape[0] * 0.90)
split_point

358

In [5]:
# Split the data
tr = shuffled_data.iloc[:split_point]
te = shuffled_data.iloc[split_point:]

# Display the training sets
print("Training Set:")
tr.head()

Training Set:


,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,33.0,4,91.0,53.0,1795,17.4,76,japan,honda civic
1,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger
2,19.0,6,232.0,100.0,2634,13.0,71,usa,amc gremlin
3,13.0,8,318.0,150.0,3940,13.2,76,usa,plymouth volare premier v8
4,14.0,8,318.0,150.0,4237,14.5,73,usa,plymouth fury gran sedan


In [6]:
# Display the testing sets
print("Testing Set:")
te.head()

Testing Set:


,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
358,21.5,3,80.0,110.0,2720,13.5,77,japan,mazda rx-4
359,35.0,4,72.0,69.0,1613,18.0,71,japan,datsun 1200
360,28.0,4,116.0,90.0,2123,14.0,71,europe,opel 1900
361,18.0,6,171.0,97.0,2984,14.5,75,usa,ford pinto
362,15.5,8,304.0,120.0,3962,13.9,76,usa,amc matador


In [7]:
# Verify that the split sizes add up to the original dataset
len(tr) + len(te) == len(data)

True

### **3.2 ~ Using Scikit-learn**

Scikit-learn provides a convenient function, `train_test_split`, to perform the train-test split.

In [8]:
# Perform train-test split using Scikit-learn
tr, te = train_test_split(data, test_size=0.1, random_state=83)

# Display the training sets
print("Training Set:")
tr.head()

Training Set:


,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
143,26.0,4,97.0,78.0,2300,14.5,74,europe,opel manta
391,36.0,4,135.0,84.0,2370,13.0,82,usa,dodge charger 2.2
182,28.0,4,107.0,86.0,2464,15.5,76,europe,fiat 131
63,14.0,8,400.0,175.0,4385,12.0,72,usa,pontiac catalina
52,30.0,4,88.0,76.0,2065,14.5,71,europe,fiat 124b


In [9]:
# Display the testing sets
print("Testing Set:")
te.head()

Testing Set:


,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
232,16.0,8,351.0,149.0,4335,14.5,77,usa,ford thunderbird
365,20.2,6,200.0,88.0,3060,17.1,81,usa,ford granada gl
225,17.5,6,250.0,110.0,3520,16.4,77,usa,chevrolet concours
277,16.2,6,163.0,133.0,3410,15.8,78,europe,peugeot 604sl
115,15.0,8,350.0,145.0,4082,13.0,73,usa,chevrolet monte carlo s


***

## **4. Quick Visualization**

Let's visualize the distribution of the `mpg` (miles per gallon) feature in both the training and testing sets.

**Note:** In practice, it's best to avoid inspecting the test data to prevent data leakage.

In [10]:
# Create a distribution plot for mpg in training and testing sets
fig = ff.create_distplot(
    [tr['mpg'].dropna(), te['mpg'].dropna()], 
    ['Train MPG', 'Test MPG'],
    bin_size=1
)
fig.show()

***

## **5. Building a Basic Model**

We will build a simple Linear Regression model using engine characteristics such as "cylinders" and "displacement" to predict `mpg`.

### **5.1 ~ Feature Extraction**

Define a function to extract the relevant features from the dataset.

In [11]:
def phi(df):
    """
    Extracts the 'cylinders' and 'displacement' features from the dataframe.
    
    Parameters:
        df (pd.DataFrame): The input dataframe.
        
    Returns:
        pd.DataFrame: Dataframe containing only the selected features.
    """
    return df[["cylinders", "displacement"]]

### **5.2 ~ Training the Model**

Fit a Linear Regression model using the extracted features.

In [12]:
# Initialize the Linear Regression model
model = LinearRegression()

# Fit the model on the training data
model.fit(phi(tr), tr['mpg'])

LinearRegression()

### **5.3 ~ Evaluating the Model**

We will use the **Root Mean Squared Error (RMSE)** to evaluate the model's performance.

RMSE provides an error metric in the same units as the target variable (`mpg`).

In [13]:
def rmse(y, yhat):
    """
    Computes the Root Mean Squared Error between actual and predicted values.
    
    Parameters:
        y (pd.Series or np.array): Actual values.
        yhat (pd.Series or np.array): Predicted values.
        
    Returns:
        float: The RMSE value.
    """
    return np.sqrt(np.mean((y - yhat) ** 2))

In [14]:
# Calculate training predictions and RMSE
Y_hat_tr = model.predict(phi(tr))
Y_tr = tr['mpg']
print("Training Error (RMSE):", rmse(Y_tr, Y_hat_tr))

Training Error (RMSE): 4.589009902639974


### **5.4 ~ Avoiding Data Leakage**

It's tempting to evaluate the model on the test set immediately, but this can lead to data leakage. Here's an example of what **not** to do:

In [15]:
# Calculate test predictions and RMSE (Not Recommended)
Y_hat_te = model.predict(phi(te))
Y_te = te['mpg']
print("Test Error (RMSE):", rmse(Y_te, Y_hat_te))

Test Error (RMSE): 5.0340093039091744


**Notice:** The test error is slightly higher than the training error, which is typical as the model may not generalize perfectly to unseen data.

***

## **6. Scikit-Learn Pipelines**

Scikit-learn's `Pipeline` allows us to streamline the workflow by chaining together preprocessing steps and the model.

This ensures that the same transformations are applied consistently during training and testing.

### **6.1 ~ Creating Pipelines**

Create a pipeline that selects specific columns and applies the Linear Regression model.

In [16]:
# Define the pipeline
pipeline = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", "passthrough", ["cylinders", "displacement"])
    ])),
    ("LinearModel", LinearRegression())
])

# Fit the pipeline on the training data
pipeline.fit(tr, tr['mpg'])

Pipeline(steps=[('SelectColumns',
                 ColumnTransformer(transformers=[('keep', 'passthrough',
                                                  ['cylinders',
                                                   'displacement'])])),
                ('LinearModel', LinearRegression())])

### **6.2 ~ Tracking Multiple Models**

To compare different models, we can store them in a dictionary with descriptive names.

In [17]:
# Initialize a dictionary to store models
models = {"cylinders_displacement": pipeline}

### **6.3 ~ Advanced Feature Transformations**

Enhance the model by adding a new feature: displacement per cylinder.

In [18]:
# Define a function to compute displacement per cylinder
def compute_volume(X):
    """
    Computes displacement per cylinder.
    
    Parameters:
        X (np.array): Array containing 'cylinders' and 'displacement'.
        
    Returns:
        np.array: Array of displacement per cylinder.
    """
    return np.expand_dims(X[:, 1] / X[:, 0], axis=1)

# Create a FunctionTransformer for the new feature
volume_transformer = FunctionTransformer(compute_volume, validate=True)

In [19]:
# Update the pipeline to include the new feature
pipeline = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", "passthrough", ["cylinders", "displacement"]),
        ("displacement_per_cylinder", volume_transformer, ["cylinders", "displacement"])
    ])),
    ("LinearModel", LinearRegression())
])

# Fit the updated pipeline
pipeline.fit(tr, tr['mpg'])

# Add the updated model to the dictionary
models["cylinders_displacement_per_cylinder"] = pipeline

In [20]:
# Evaluate the updated model on the training data
Y_hat_tr = pipeline.predict(tr)
print("Training Error (RMSE):", rmse(tr['mpg'], Y_hat_tr))

Training Error (RMSE): 4.318799994795116


### **6.4 ~ Handling Missing Values**

The dataset contains missing values, particularly in the `horsepower` column.

We will handle these by imputing missing values with the mean of the respective columns.

In [21]:
# Define quantitative features
quantitative_features = ["cylinders", "displacement", "horsepower", "weight", "acceleration"]

In [22]:
# Update the pipeline to include more features and handle missing values
pipeline = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", "passthrough", quantitative_features),
        ("displacement_per_cylinder", volume_transformer, ["cylinders", "displacement"])
    ])),
    ("Imputation", SimpleImputer()),
    ("LinearModel", LinearRegression())
])

# Attempt to fit the pipeline (expected to raise an error due to missing values)
try:
    pipeline.fit(tr, tr['mpg'])
except ValueError as err:
    print("Error:", err)

In [23]:
# Fit the pipeline with imputation
pipeline.fit(tr, tr['mpg'])

# Add the model with imputation to the dictionary
models['cylinders_horsepower_weight_acceleration'] = pipeline

# Evaluate the updated model on the training data
Y_hat_tr = pipeline.predict(tr)
print("Training Error (RMSE):", rmse(tr['mpg'], Y_hat_tr))

Training Error (RMSE): 4.025034093127813


***

## **7. Cross-Validation**

Cross-validation provides a more robust evaluation of the model's performance by training and validating it on different subsets of the data.

### **7.1 ~ Implementing Cross-Validation**

Define a function to perform K-Fold cross-validation and compute the average RMSE.

In [24]:
def cross_validate_rmse(model, X, y, n_splits=5):
    """
    Performs K-Fold cross-validation and computes the average RMSE.
    
    Parameters:
        model (sklearn Pipeline): The model pipeline to evaluate.
        X (pd.DataFrame): Features.
        y (pd.Series): Target variable.
        n_splits (int): Number of folds.
        
    Returns:
        float: Average RMSE across folds.
    """
    model = clone(model)
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    rmse_values = []
    
    for tr_ind, va_ind in kfold.split(X):
        X_train, X_val = X.iloc[tr_ind], X.iloc[va_ind]
        y_train, y_val = y.iloc[tr_ind], y.iloc[va_ind]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        rmse_values.append(rmse(y_val, y_pred))
    
    return np.mean(rmse_values)

In [25]:
# Validate the latest model using cross-validation
cv_rmse = cross_validate_rmse(pipeline, tr, tr['mpg'])
print("Cross-Validation RMSE:", cv_rmse)

Cross-Validation RMSE: 4.139401940486051


### **7.2 ~ Comparing Models**

Define a helper function to compare all models stored in the `models` dictionary.

In [26]:
def compare_models(models, tr, te):
    """
    Compares multiple models by computing their training RMSE, cross-validation RMSE, and test RMSE.
    
    Parameters:
        models (dict): Dictionary of models with their names as keys.
        tr (pd.DataFrame): Training dataset.
        te (pd.DataFrame): Testing dataset.
        
    Returns:
        plotly.graph_objects.Figure: A bar chart comparing RMSEs.
    """
    training_rmse = [rmse(tr['mpg'], model.predict(tr)) for model in models.values()]
    validation_rmse = [cross_validate_rmse(model, tr, tr['mpg']) for model in models.values()]
    test_rmse = [rmse(te['mpg'], model.predict(te)) for model in models.values()]
    names = list(models.keys())
    
    fig = go.Figure([
        go.Bar(x=names, y=training_rmse, name="Training RMSE"),
        go.Bar(x=names, y=validation_rmse, name="Cross-Validation RMSE"),
        go.Bar(x=names, y=test_rmse, name="Test RMSE", opacity=0.6)
    ])
    
    fig.update_layout(
        title="Model RMSE Comparison",
        yaxis_title="RMSE",
        barmode='group'
    )
    return fig

In [27]:
# Generate a comparison plot for all models
fig = compare_models(models, tr, te)
fig.update_yaxes(range=[0, 10], title="RMSE")
fig.show()

**Note:** The test RMSE is plotted with reduced opacity to discourage reliance on it before final model evaluation.

***

## **8. Enhancing the Model**

Let's further improve the model by adding the `model_year` feature.

In [28]:
# Update the list of quantitative features to include 'model_year'
quantitative_features = ["cylinders", "displacement", "horsepower", "weight", "acceleration", "model_year"]

In [29]:
# Update the pipeline to include 'model_year'
pipeline = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", "passthrough", quantitative_features),
        ("displacement_per_cylinder", volume_transformer, ["cylinders", "displacement"])
    ])),
    ("Imputation", SimpleImputer()),
    ("LinearModel", LinearRegression())
])

# Fit the updated pipeline
pipeline.fit(tr, tr['mpg'])

# Add the enhanced model to the dictionary
models['cylinders_model_year'] = pipeline

In [30]:
# Compare all models after adding 'model_year'
fig = compare_models(models, tr, te)
fig.update_yaxes(range=[0, 10], title="RMSE")
fig.show()

**Observation:** Adding `model_year` has improved both training and cross-validation RMSE, indicating better model performance and generalization.

***

## **9. Potential Overfitting**

Can we push the model further by using the car's name to predict `mpg`? While incorporating textual data like the car's name might introduce useful features (e.g., manufacturer), it can also lead to overfitting due to high dimensionality and specific information not present in the training set.

In [31]:
# Define the pipeline with text features
pipeline_with_text = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("keep", "passthrough", quantitative_features),
        ("displacement_per_cylinder", volume_transformer, ["cylinders", "displacement"]),
        ("car_name", CountVectorizer(), "name")
    ])),
    ("Imputation", SimpleImputer(strategy='mean')),
    ("LinearModel", LinearRegression())
])

# Fit the pipeline with text features
pipeline_with_text.fit(tr, tr['mpg'])

# Add the overextended model to the dictionary
models['cylinders_overextended'] = pipeline_with_text

In [32]:
# Compare all models including the overextended one
fig = compare_models(models, tr, te)
fig.update_yaxes(range=[0, 20], title="RMSE")
fig.show()

**Result:** Including the car's name significantly reduces training RMSE but increases cross-validation and test RMSE, indicating overfitting.